# 1 feature extraction

# 1.1 Habitat Feature Extraction

In [ ]:
import os
import pandas as pd
from radiomics import featureextractor
import SimpleITK as sitk

# 从YAML文件读取配置参数
params_path = 'C:/Radiomics analysis/live.yaml'
extractor = featureextractor.RadiomicsFeatureExtractor(params_path)

# 文件夹路径
roi_folder_path = 'C:/Users/21581/Desktop/AllRadiomics_feature/ICCfiles/ICC20live'  # ROI文件夹路径
image_folder_path = 'C:/Users/21581/Desktop/AllRadiomics_feature/ICCfiles/ICC20'  # 原始图像文件夹路径
output_folder = 'C:/Users/21581/Desktop/AllRadiomics_feature/ICCfiles/ICC20livefeature'  # 输出文件夹路径
error_log_path = os.path.join(output_folder, 'error_log.csv')  # 错误记录文件路径

# 确保输出文件夹存在
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 初始化错误记录列表
error_log = []

# 为每个label初始化一个空的DataFrame
feature_dfs = {label_num: pd.DataFrame() for label_num in range(1, 10)}

# 遍历原始图像文件夹的子文件夹，每个子文件夹以患者姓名命名
for patient_name in os.listdir(image_folder_path):
    patient_folder = os.path.join(image_folder_path, patient_name)
    
    # 检查路径是否为目录
    if not os.path.isdir(patient_folder):
        continue
    
    # 构建图像文件路径
    image_file_path = os.path.join(patient_folder, 'image_path.nii')
    
    if not os.path.isfile(image_file_path):
        print(f'Image file not found for patient: {patient_name}')
        continue
    
    # 加载原始图像
    image = sitk.ReadImage(image_file_path)
    
    for label_num in range(1, 10):
        # 构建ROI文件的完整路径
        roi_file_path = os.path.join(roi_folder_path, f'{patient_name}_habitat.nii.gz')
        
        if not os.path.isfile(roi_file_path):
            print(f'ROI file not found for patient: {patient_name}, label: {label_num}')
            continue
        
        try:
            # 加载ROI标签图像
            label_image = sitk.ReadImage(roi_file_path)
            binary_label = sitk.BinaryThreshold(label_image, lowerThreshold=label_num, upperThreshold=label_num, insideValue=1, outsideValue=0)
            
            # 提取特征
            features = extractor.execute(image, binary_label)
            features_df = pd.Series(features)
            
            # 将患者姓名作为索引添加到DataFrame
            features_df.name = patient_name
            
            # 将特征数据添加到相应label的DataFrame
            feature_dfs[label_num] = feature_dfs[label_num].append(features_df)
            
            print(f'Extracted features for patient: {patient_name}, label: {label_num}')
        except Exception as e:
            print(f'Failed to extract features for patient: {patient_name}, label: {label_num}. Error: {e}')
            # 记录发生错误的文件
            error_log.append({'Patient': patient_name, 'Label': label_num, 'Error': str(e)})

# 保存每个label的特征到CSV文件
for label_num, df in feature_dfs.items():
    output_file_path = os.path.join(output_folder, f'features_label_{label_num}.csv')
    df.to_csv(output_file_path)
    print(f'Saved features for label {label_num} to {output_file_path}')

# 保存错误日志到CSV文件
error_log_df = pd.DataFrame(error_log)
error_log_df.to_csv(error_log_path, index=False)
print(f'Saved error log to {error_log_path}')


# 1.2 Peritumor&Primary Tumor Feature Extraction

In [ ]:
#240316瘤周特征提取代码
import os
import csv
import pandas as pd
from radiomics import featureextractor

# 瘤周特征提取
image_dir = 'C:/Users/21581/Desktop/AllRadiomics_feature/ICCperitumor/1mm/image'
roi_dir = 'C:/Users/21581/Desktop/AllRadiomics_feature/ICCperitumor/1mm/ROI'
params_path = 'C:/Radiomics analysis/Peritumor.yaml'

extractor = featureextractor.RadiomicsFeatureExtractor(params_path)

patient_features = {}
patients_with_no_labels = []  # 用于存储没有标签的病人ID

for image_file in os.listdir(image_dir):
    if image_file.endswith('.nii'):
        patient_id = image_file.rsplit('_', 1)[0]
        image_path = os.path.join(image_dir, image_file)
        roi_file = f'{patient_id}_roi.nii'
        roi_path = os.path.join(roi_dir, roi_file)
        if not os.path.exists(roi_path):
            print(f'警告：未找到 {patient_id} 的ROI文件。')
            patients_with_no_labels.append(patient_id)
            continue  # 如果没有找到ROI文件，则跳过这个病人

        try:
            print(f'正在处理 {patient_id} 的特征提取...')
            features = extractor.execute(image_path, roi_path)
            patient_features[patient_id] = features
        except Exception as e:
            if 'No labels found' in str(e):
                print(f'警告：{patient_id} 的掩膜中没有找到标签。')
                patients_with_no_labels.append(patient_id)
            else:
                print(f'警告：处理 {patient_id} 时出现其他错误。错误信息：{e}')

# 导出没有标签的病人列表到CSV文件
if patients_with_no_labels:
    df_no_labels = pd.DataFrame(patients_with_no_labels, columns=['PatientID'])
    df_no_labels.to_csv('C:/Users/21581/Desktop/patients_with_no_labels.csv', index=False)
    print('导出没有掩膜标签的患者列表到CSV文件。')

# 导出特征到CSV文件
if patient_features:
    output_path = 'C:/Users/21581/Desktop/output_features.csv'
    with open(output_path, 'w', newline='') as csvfile:
        fieldnames = ['PatientID'] + list(next(iter(patient_features.values())).keys())
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for patient_id, features in patient_features.items():
            row = {'PatientID': patient_id}
            row.update(features)
            writer.writerow(row)
    print(f"特征提取完毕，并已导出到CSV文件：{output_path}")
else:
    print("没有特征被提取，请检查文件路径和格式。")

# 2 feature preprocessing

# BorderlineSMOTE

In [ ]:
# BorderlineSMOTE 重采样拆分
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import BorderlineSMOTE  # 修改这里导入BorderlineSMOTE
import os

# 设定CSV文件所在的目录路径
directory_path = 'C:/Users/21581/Desktop/analysis0430'
output_path = 'C:/Users/21581/Desktop/analysis0430/trainandtest'
if not os.path.exists(output_path):
    os.makedirs(output_path)  # 如果输出目录不存在，创建它

# CSV文件列表
csv_files = [
    'filtered0mm.csv',
    'filtered1mm.csv',
    'filtered2mm.csv',
    'filtered3mm.csv',
    'filtered4mm.csv',
    'filtered5mm.csv',
    'CTCclinical.csv',
    'live.csv',
]

for csv_file in csv_files:
    df = pd.read_csv(os.path.join(directory_path, csv_file), encoding='ISO-8859-1')

    # 分离特征和标签
    X = df.iloc[:, 2:]  # 特征列
    y = df.iloc[:, 1]   # 标签列
    patient_info = df.iloc[:, :1]  # 患者信息列
    
    # 拆分数据集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=420)
    
    # 应用BorderlineSMOTE进行过采样
    smote = BorderlineSMOTE(random_state=420)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    
    # 创建与重采样后数据相对应的患者信息，新样本填充默认值 'aaaa'
    patient_info_resampled = pd.DataFrame('aaaa', index=X_train_resampled.index, columns=['info'])
    # 更新原始样本的患者信息
    patient_info_resampled.loc[X_train.index, 'info'] = patient_info.loc[X_train.index].squeeze()

    # 初始化标准化器
    scaler = StandardScaler()
    
    # 标准化训练数据和测试数据
    X_train_scaled = scaler.fit_transform(X_train_resampled)
    X_test_scaled = scaler.transform(X_test)
    
    # 将标准化后的数据转换回DataFrame
    X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train_resampled.columns, index=X_train_resampled.index)
    X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)
    
    # 合并患者信息、标准化后的特征和标签
    train_data = pd.concat([patient_info_resampled, X_train_scaled, y_train_resampled], axis=1)
    test_data = pd.concat([patient_info.loc[X_test.index], X_test_scaled, y_test], axis=1)
    
    # 保存到CSV文件
    train_output_file = os.path.join(output_path, f'train_{csv_file}')
    test_output_file = os.path.join(output_path, f'test_{csv_file}')
    train_data.to_csv(train_output_file, index=False)
    test_data.to_csv(test_output_file, index=False)
    print(f'Processed {csv_file}: Train and test CSV files saved.')


# Univariate analysis

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.stats import mannwhitneyu, ttest_ind, shapiro

# 输入文件夹和输出文件夹的路径
input_folder = 'C:/Users/21581/Desktop/240517live_selection/bspline88/subtumor_data'
output_folder = 'C:/Users/21581/Desktop/240517live_selection/bspline88/subtumor_data/p_value_filtered'

# 如果输出文件夹不存在，创建输出文件夹
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 准备记录删除特征数量的列表
features_removed = []

def process_files(train_file, test_file):
    try:
        # 读取训练集和测试集文件
        train_df = pd.read_csv(os.path.join(input_folder, train_file))
        test_df = pd.read_csv(os.path.join(input_folder, test_file))
        
        # 假定第一列是标签
        y = train_df.iloc[:, 0]
        p_values = []
        
        # 对每个特征进行正态性检验和适当的统计测试
        for col in train_df.columns[1:]:  # 排除第一列标签
            # 正态性检验
            p_normal_train = shapiro(train_df[train_df.iloc[:, 0] == 1][col])[1]
            p_normal_test = shapiro(train_df[train_df.iloc[:, 0] == 0][col])[1]
            
            # 检验是否满足正态分布
            if p_normal_train > 0.05 and p_normal_test > 0.05:
                # 使用t检验
                stat, p = ttest_ind(train_df[train_df.iloc[:, 0] == 1][col], 
                                    train_df[train_df.iloc[:, 0] == 0][col])
            else:
                # 使用Mann-Whitney U检验
                stat, p = mannwhitneyu(train_df[train_df.iloc[:, 0] == 1][col], 
                                       train_df[train_df.iloc[:, 0] == 0][col])
            p_values.append(p)
        
        # 将p值转换为Series对象，并筛选p值小于0.05的特征
        p_values_series = pd.Series(p_values, index=train_df.columns[1:])
        to_keep = p_values_series[p_values_series < 0.05].index.tolist()
        
        # 记录删除的特征数量
        removed_features_count = len(train_df.columns[1:]) - len(to_keep)
        features_removed.append({'file': train_file, 'removed_features_count': removed_features_count})
        
        # 保留p值小于0.05的特征和标签列
        train_df_reduced = train_df[['Pathology'] + to_keep]
        test_df_reduced = test_df[['Pathology'] + to_keep]
        
        # 保存筛选后的数据集
        train_df_reduced.to_csv(os.path.join(output_folder, f'reduced_{train_file}'), index=False)
        test_df_reduced.to_csv(os.path.join(output_folder, f'reduced_{test_file}'), index=False)
        
    except Exception as e:
        print(f"处理{train_file}时出错：{e}")

# 训练集和测试集文件名列表
# 训练集和测试集文件名列表

train_files = ['train_processed_features_label_1.csv', 'train_processed_features_label_2.csv', 'train_processed_features_label_3.csv',
               'train_processed_features_label_4.csv', 'train_processed_features_label_5.csv', 'train_processed_features_label_6.csv',
               'train_processed_features_label_7.csv', 'train_processed_features_label_8.csv', 'train_processed_features_label_9.csv']
test_files = ['test_processed_features_label_1.csv', 'test_processed_features_label_2.csv', 'test_processed_features_label_3.csv',
              'test_processed_features_label_4.csv', 'test_processed_features_label_5.csv', 'test_processed_features_label_6.csv',
              'test_processed_features_label_7.csv', 'test_processed_features_label_8.csv', 'test_processed_features_label_9.csv']


# 处理每个文件对
for train_file, test_file in zip(train_files, test_files):
    process_files(train_file, test_file)

# 将删除的特征数量的数据保存为CSV文件
features_removed_df = pd.DataFrame(features_removed)
features_removed_df.to_csv(os.path.join(output_folder, 'removed_features_summary.csv'), index=False)




# Spearman Correlation Analysis

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

# 输入文件夹和输出文件夹的路径
input_folder = 'C:/Users/21581/Desktop/240517live_selection/bspline88/subtumor_data/p_value_filtered'
output_folder = 'C:/Users/21581/Desktop/240517live_selection/bspline88/subtumor_data/spearman'

# 如果输出文件夹不存在，创建输出文件夹
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 用于记录每个文件删除特征数量的字典
features_removed = {}

def process_files(train_file, test_file):
    try:
        # 读取训练集和测试集文件
        train_df = pd.read_csv(os.path.join(input_folder, train_file))
        test_df = pd.read_csv(os.path.join(input_folder, test_file))
        
        # 计算Spearman秩相关性矩阵
        corr_matrix = train_df.corr(method='spearman').abs()
        
        # 选择相关性矩阵的上三角部分，避免重复特征对
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        
        # 找到相关系数大于0.75的特征列
        to_drop = [column for column in upper.columns if any(upper[column] > 0.75)]
        
        # 记录删除特征的数量
        features_removed[train_file] = len(to_drop)
        
        # 删除这些列
        train_df_reduced = train_df.drop(columns=to_drop)
        test_df_reduced = test_df.drop(columns=to_drop)
        
        # 保存筛选后的数据集
        train_df_reduced.to_csv(os.path.join(output_folder, f'reduced_{train_file}'), index=False)
        test_df_reduced.to_csv(os.path.join(output_folder, f'reduced_{test_file}'), index=False)
        
        # 如果需要，生成并保存相关性热图
        if os.environ.get('GENERATE_HEATMAPS', '0') == '1':
            plt.figure(figsize=(20, 16))
            sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
            plt.title(f'{train_file}的相关性矩阵')
            plt.savefig(os.path.join(output_folder, f'heatmap_{train_file}.png'))
            plt.close()
    except Exception as e:
        print(f"处理{train_file}时出错：{e}")
    
# 训练集和测试集文件名列表
    
train_files = ['reduced_train_processed_features_label_1.csv', 'reduced_train_processed_features_label_2.csv', 'reduced_train_processed_features_label_3.csv',
               'reduced_train_processed_features_label_4.csv', 'reduced_train_processed_features_label_5.csv', 'reduced_train_processed_features_label_6.csv',
               'reduced_train_processed_features_label_7.csv', 'reduced_train_processed_features_label_8.csv', 'reduced_train_processed_features_label_9.csv']
test_files = ['reduced_test_processed_features_label_1.csv', 'reduced_test_processed_features_label_2.csv', 'reduced_test_processed_features_label_3.csv',
              'reduced_test_processed_features_label_4.csv', 'reduced_test_processed_features_label_5.csv', 'reduced_test_processed_features_label_6.csv',
              'reduced_test_processed_features_label_7.csv', 'reduced_test_processed_features_label_8.csv', 'reduced_test_processed_features_label_9.csv']
    
    
# 处理每个文件对
for train_file, test_file in zip(train_files, test_files):
    process_files(train_file, test_file)
    
# 将删除特征的数量的数据保存为CSV文件
features_removed_df = pd.DataFrame(list(features_removed.items()), columns=['File', 'Removed Features Count'])
features_removed_df.to_csv(os.path.join(output_folder, 'removed_features_summary_spearman.csv'), index=False)


# 3 Model Construction

# Stacking model

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, roc_curve
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import StackingClassifier
from skopt import BayesSearchCV
from skopt.space import Integer, Categorical
import warnings
import matplotlib.pyplot as plt

# Suppress warnings
warnings.filterwarnings('ignore')

# 设置随机种子
RANDOM_SEED = 5

# 数据集路径
input_dir = 'C:/Users/21581/Desktop/240517live_selection/bspline88/subtumor_data/combine/1'
output_dir = 'C:/Users/21581/Desktop/240517live_selection/bspline88/subtumor_data/combine/1/11'

# 确保输出目录存在
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 获取训练和测试文件路径
train_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if 'train' in f and f.endswith('.csv')]
test_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if 'test' in f and f.endswith('.csv')]

X_trains, y_trains, X_tests, y_tests = [], [], [], []

# 加载数据集
for train_path, test_path in zip(train_files, test_files):
    train_df = pd.read_csv(train_path)
    test_df = pd.read_csv(test_path)
    X_trains.append(train_df.drop('Pathology', axis=1).values)
    y_trains.append(train_df['Pathology'].values)
    X_tests.append(test_df.drop('Pathology', axis=1).values)
    y_tests.append(test_df['Pathology'].values)

# 确保训练和测试数据集长度一致
def ensure_same_length(X_list, y_list):
    min_length = min(len(X) for X in X_list)
    X_list = [X[:min_length] for X in X_list]
    y_list = [y[:min_length] for y in y_list]
    return X_list, y_list

X_trains, y_trains = ensure_same_length(X_trains, y_trains)
X_tests, y_tests = ensure_same_length(X_tests, y_tests)

# 基础模型列表，增加正则化强度，并添加更多基础模型
base_models = [
    ('lr', LogisticRegression(max_iter=100, penalty='l2', C=0.001, random_state=RANDOM_SEED)),  # 增加正则化
    ('rf', RandomForestClassifier(n_estimators=30, max_depth=2, random_state=RANDOM_SEED)),     # 减少模型复杂度
    ('svc', SVC(probability=True, C=0.001, kernel='linear', random_state=RANDOM_SEED)),         # 增加正则化
    ('knn', KNeighborsClassifier(n_neighbors=20)),                                             # 增加K值，减少过拟合
    ('gbc', GradientBoostingClassifier(n_estimators=30, learning_rate=0.05, random_state=RANDOM_SEED))  # 减少复杂度
]

# 使用交叉验证生成元特征
def get_meta_features(model, X_trains, y_trains, X_tests, n_splits=5, n_repeats=10):
    n_samples = X_trains[0].shape[0]
    meta_train = np.zeros((n_samples,))
    meta_test_list = []
    rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=RANDOM_SEED)
    
    for train_index, val_index in rskf.split(X_trains[0], y_trains[0]):
        fold_meta_train = np.zeros((len(val_index),))
        fold_meta_test = np.zeros((len(X_tests[0]), len(X_tests)))
        for i, (X_train, y_train, X_test) in enumerate(zip(X_trains, y_trains, X_tests)):
            X_tr, X_val = X_train[train_index], X_train[val_index]
            y_tr, y_val = y_train[train_index], y_train[val_index]
            model.fit(X_tr, y_tr)
            fold_meta_train += model.predict_proba(X_val)[:, 1]
            fold_meta_test[:, i] = model.predict_proba(X_test)[:, 1]
        
        meta_train[val_index] = fold_meta_train / len(X_trains)
        meta_test_list.append(np.mean(fold_meta_test, axis=1))
    
    meta_test = np.mean(meta_test_list, axis=0)
    
    # 检查模型是否选择了特征
    selected_features = False
    if hasattr(model, 'coef_'):
        selected_features = np.any(model.coef_ != 0)
    elif hasattr(model, 'feature_importances_'):
        selected_features = np.any(model.feature_importances_ != 0)
    
    return meta_train, meta_test, selected_features

# 初始化元特征数组
meta_train_list = []
meta_test_list = []
selected_models = []

for name, model in base_models:
    meta_train, meta_test, selected_features = get_meta_features(model, X_trains, y_trains, X_tests)
    meta_train_list.append(meta_train)
    meta_test_list.append(meta_test)
    if selected_features:
        selected_models.append(name)

# 汇总元特征
meta_train = np.column_stack(meta_train_list)
meta_test = np.column_stack(meta_test_list)

# 保存元特征到CSV文件
pd.DataFrame(meta_train).to_csv(os.path.join(output_dir, 'meta_features_train.csv'), index=False)
pd.DataFrame(meta_test).to_csv(os.path.join(output_dir, 'meta_features_test.csv'), index=False)

# 打印哪些模型选择了特征
print("Selected models with features:", selected_models)

# 构建StackingClassifier，使用LogisticRegression作为元模型并进行贝叶斯优化
stacking_model = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression(random_state=RANDOM_SEED, penalty='l2', C=0.01),
    passthrough=True
)

# 定义贝叶斯优化的参数空间
param_space = {
    'final_estimator__C': [0.0001, 0.001, 0.01, 0.1, 1],
    'final_estimator__penalty': ['l1', 'l2'],
    'final_estimator__solver': ['liblinear', 'saga']
}

# 使用BayesSearchCV进行贝叶斯优化
bayes_search = BayesSearchCV(
    estimator=stacking_model,
    search_spaces=param_space,
    n_iter=50,
    cv=RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=RANDOM_SEED),
    scoring='roc_auc',
    n_jobs=-1,
    verbose=0,
    random_state=RANDOM_SEED
)

# 训练StackingClassifier
bayes_search.fit(meta_train, y_trains[0])

best_model = bayes_search.best_estimator_

# 预测
train_probabilities = best_model.predict_proba(meta_train)[:, 1]
test_probabilities = best_model.predict_proba(meta_test)[:, 1]

# 计算ROC曲线数据
fpr_train, tpr_train, _ = roc_curve(y_trains[0], train_probabilities)
fpr_test, tpr_test, _ = roc_curve(y_tests[0], test_probabilities)

# 绘制ROC曲线
plt.figure()
plt.plot(fpr_train, tpr_train, label='Train ROC curve (area = %0.2f)' % roc_auc_score(y_trains[0], train_probabilities))
plt.plot(fpr_test, tpr_test, label='Test ROC curve (area = %0.2f)' % roc_auc_score(y_tests[0], test_probabilities))
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

# 打印评估指标
y_train_pred = (train_probabilities > 0.5).astype(int)
y_test_pred = (test_probabilities > 0.5).astype(int)

print("Train Accuracy: ", accuracy_score(y_trains[0], y_train_pred))
print("Test Accuracy: ", accuracy_score(y_tests[0], y_test_pred))
print("Train F1 Score: ", f1_score(y_trains[0], y_train_pred))
print("Test F1 Score: ", f1_score(y_tests[0], y_test_pred))


# Stacking Model 2

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, f1_score, precision_recall_curve
import matplotlib.pyplot as plt

# 加载第一层的元特征矩阵
meta_train = pd.read_csv('C:/Users/21581/Desktop/240517live_selection/bspline88/subtumor_data/results/predicted_results/meta_featuressss/meta_features_train.csv')
meta_test = pd.read_csv('C:/Users/21581/Desktop/240517live_selection/bspline88/subtumor_data/results/predicted_results/meta_featuressss/meta_features_test.csv')

# 加载临床特征数据
train_clinical_df = pd.read_csv('C:/Users/21581/Desktop/240517live_selection/bspline88/subtumor_data/results/predicted_results/meta_featuressss/trainCTCCTC.csv')  # 示例路径
test_clinical_df = pd.read_csv('C:/Users/21581/Desktop/240517live_selection/bspline88/subtumor_data/results/predicted_results/meta_featuressss/testCTCCTC.csv')  # 示例路径

# 获取标签
y_train = train_clinical_df['Pathology'].values
y_test = test_clinical_df['Pathology'].values

# 融合元特征和临床特征
X_train_combined = np.column_stack((meta_train.values, train_clinical_df[['Tumor_diameter', 'CTC']].values))
X_test_combined = np.column_stack((meta_test.values, test_clinical_df[['Tumor_diameter', 'CTC']].values))

# 使用 StackingClassifier 作为第一层模型
second_layer_model = StackingClassifier(
    estimators=[
        ('lr', LogisticRegression()),
        ('svc', SVC(probability=True)),
        ('knn', KNeighborsClassifier()),
        ('gbc', GradientBoostingClassifier())
    ],
    final_estimator=LogisticRegression(penalty='l1', solver='liblinear', C=1.0),  # 使用L1正则化的Logistic回归
    passthrough=True,
    cv=5
)
second_layer_model.fit(X_train_combined, y_train)

# 预测并评估训练集
train_predictions = second_layer_model.predict_proba(X_train_combined)[:, 1]
train_auc = roc_auc_score(y_train, train_predictions)
train_fpr, train_tpr, _ = roc_curve(y_train, train_predictions)
train_accuracy = accuracy_score(y_train, (train_predictions > 0.5).astype(int))

# 预测并评估测试集
test_predictions = second_layer_model.predict_proba(X_test_combined)[:, 1]
test_auc = roc_auc_score(y_test, test_predictions)
test_fpr, test_tpr, _ = roc_curve(y_test, test_predictions)
test_accuracy = accuracy_score(y_test, (test_predictions > 0.5).astype(int))

# 寻找最佳阈值以优化F1 Score
precision, recall, thresholds = precision_recall_curve(y_test, test_predictions)
f1_scores = 2 * precision * recall / (precision + recall)
best_threshold = thresholds[np.argmax(f1_scores)]
test_f1 = np.max(f1_scores)

# 使用最佳阈值重新计算F1 Score
train_f1 = f1_score(y_train, (train_predictions > best_threshold).astype(int))
test_f1 = f1_score(y_test, (test_predictions > best_threshold).astype(int))

# 打印评估指标
print(f"Train ROC AUC: {train_auc:.2f}")
print(f"Test ROC AUC: {test_auc:.2f}")
print(f"Train Accuracy: {train_accuracy:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Train F1 Score: {train_f1:.2f}")
print(f"Test F1 Score: {test_f1:.2f}")

# 绘制ROC曲线
plt.figure()

# 绘制每条曲线并添加数据点
plt.plot(train_fpr, train_tpr, color='red', label=f'Training Dataset AUC={train_auc:.2f}')
plt.scatter(train_fpr, train_tpr, color='red', s=10)  # s参数控制点的大小
plt.plot(test_fpr, test_tpr, color='orange', label=f'Test Dataset AUC={test_auc:.2f}')
plt.scatter(test_fpr, test_tpr, color='orange', s=10)  # s参数控制点的大小

# 绘制对角线
plt.plot([0, 1], [0, 1], 'k--')

# 设置图形标签
plt.xlabel('False positive rate (1-specificity)')
plt.ylabel('True positive rate (sensitivity)')
plt.title('ROC Curve')

# 设置图例位置
plt.legend(loc="lower right")

# 显示图形
plt.show()


# ROC AUC Curve

In [ ]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
from sklearn.metrics import roc_auc_score

# 定义文件路径和自定义标签
file_paths = [
    'C:/Users/21581/Desktop/ROCPKL/combinelivetest.pkl',
    'C:/Users/21581/Desktop/ROCPKL/combine0mmtest.pkl',
    'C:/Users/21581/Desktop/ROCPKL/combine1mmtest.pkl',
    'C:/Users/21581/Desktop/ROCPKL/combine3mmtest.pkl',
    'C:/Users/21581/Desktop/ROCPKL/combine5mmtest.pkl',
    'C:/Users/21581/Desktop/ROCPKL/onlyCTCtest.pkl',
    'C:/Users/21581/Desktop/ROCPKL/onlylivetest.pkl',
]
labels = [
    'Habitat_CTC',
    'Primary Lesion',
    'Prei_1mm',
    'Prei_3mm',
    'Prei_5mm',
    'CTC_Clinical',
    'Habitat',
]

# 初始化列表存储每个pkl文件的内容
roc_data = []

# 加载每个pkl文件
for path in file_paths:
    roc_data.append(joblib.load(path))

# 计算AUC的95%置信区间
def auc_confidence_interval(y_true, y_scores, confidence_level=0.95):
    auc = roc_auc_score(y_true, y_scores)
    n1 = sum(y_true == 1)
    n2 = sum(y_true == 0)
    q1 = auc / (2 - auc)
    q2 = 2 * auc ** 2 / (1 + auc)
    auc_var = (auc * (1 - auc) +
               (n1 - 1) * (q1 - auc ** 2) +
               (n2 - 1) * (q2 - auc ** 2)) / (n1 * n2)
    auc_std = np.sqrt(auc_var)
    z = stats.norm.ppf(1 - (1 - confidence_level) / 2)
    lower_bound = auc - z * auc_std
    upper_bound = auc + z * auc_std
    return auc, lower_bound, upper_bound

# 绘制ROC曲线
plt.figure()

# 遍历加载的ROC数据，并绘制每条曲线
for idx, (fpr, tpr, auc, y_true, y_scores) in enumerate(roc_data):
    # 计算AUC的95%置信区间
    auc, lower_bound, upper_bound = auc_confidence_interval(y_true, y_scores)
    label = f'{labels[idx]} AUC={auc:.2f} (95% CI: {lower_bound:.2f} - {upper_bound:.2f})'
    plt.plot(fpr, tpr, label=label)

# 绘制对角线
plt.plot([0, 1], [0, 1], 'k--')

# 设置图形标签
plt.xlabel('False positive rate (1-specificity)')
plt.ylabel('True positive rate (sensitivity)')
plt.title('Combined ROC Curve')

# 设置图例位置
plt.legend(loc="lower right")

# 保存图形到指定路径
save_path = 'C:/Users/21581/Desktop/ROCPKL/combined_roc_curve241130.pdf'
plt.savefig(save_path)

# 显示图形
plt.show()


# DCA Curve

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.metrics import confusion_matrix

# 你提供的函数
def calculate_net_benefit_model(thresh_group, y_pred_score, y_label):
    net_benefit_model = np.array([])
    for thresh in thresh_group:
        if thresh == 0 or thresh == 1:
            net_benefit_model = np.append(net_benefit_model, 0)
            continue
        y_pred_label = y_pred_score > thresh
        tn, fp, fn, tp = confusion_matrix(y_label, y_pred_label).ravel()
        n = len(y_label)
        net_benefit = (tp / n) - (fp / n) * (thresh / (1 - thresh))
        net_benefit_model = np.append(net_benefit_model, net_benefit)
    return net_benefit_model

def calculate_net_benefit_all(thresh_group, y_label):
    net_benefit_all = np.array([])
    tn, fp, fn, tp = confusion_matrix(y_label, y_label).ravel()
    total = tp + tn
    for thresh in thresh_group:
        if thresh == 0 or thresh == 1:
            net_benefit_all = np.append(net_benefit_all, 0)
            continue
        net_benefit = (tp / total) - (tn / total) * (thresh / (1 - thresh))
        net_benefit_all = np.append(net_benefit_all, net_benefit)
    return net_benefit_all

def plot_DCA(ax, thresh_group, net_benefit_model, label, add_treat_labels=False):
    # Plot
    ax.plot(thresh_group, net_benefit_model, label=f'{label}')

    # Fill，显示出模型较于treat all和treat none好的部分
    y2 = np.maximum(calculate_net_benefit_all(thresh_group, y_true), 0)
    y1 = np.maximum(net_benefit_model, y2)
    ax.fill_between(thresh_group, y1, y2, alpha=0.2)

    return ax

# 加载测试集的pkl文件，假设每个模型的文件路径不同
model_paths = [
    'C:/Users/21581/Desktop/ROCPKL/combinelivetrain.pkl',
    # 'C:/Users/21581/Desktop/ROCPKL/combine0mmtest.pkl',
    # 'C:/Users/21581/Desktop/ROCPKL/combine1mmtest.pkl',
    # 'C:/Users/21581/Desktop/ROCPKL/combine3mmtest.pkl',
    # 'C:/Users/21581/Desktop/ROCPKL/combine5mmtest.pkl',
    'C:/Users/21581/Desktop/ROCPKL/onlyCTCtrain.pkl',
    'C:/Users/21581/Desktop/ROCPKL/onlylivetrain.pkl',
]

# 自定义模型名称
model_names = [
    'Habitat_CTC',
    'CTC',
    'Habitat'
]

# 读取所有模型的数据
models_data = [joblib.load(path) for path in model_paths]

# 设置阈值范围
thresholds = np.linspace(0, 1, 100)

# 初始化图形
fig, ax = plt.subplots(figsize=(10, 6))

# 计算并绘制每个模型的DCA曲线
for i, (y_true, predictions) in enumerate([(data[3], data[4]) for data in models_data]):
    net_benefit_model = calculate_net_benefit_model(thresholds, predictions, y_true)
    add_treat_labels = (i == 0)
    ax = plot_DCA(ax, thresholds, net_benefit_model, label=model_names[i], add_treat_labels=add_treat_labels)

# 绘制 Treat all 和 Treat none 的基准线
net_benefit_all = calculate_net_benefit_all(thresholds, y_true)
ax.plot(thresholds, net_benefit_all, color='black', linestyle='--', label='Treat all')
ax.plot((0, 1), (0, 0), color='black', linestyle=':', label='Treat none')

# 图形配置
ax.set_xlim(0, 1)
ax.set_ylim(-0.15, net_benefit_model.max() + 0.15)
ax.set_xlabel('Threshold Probability', fontdict={'family': 'Arial', 'fontsize': 16})
ax.set_ylabel('Net Benefit', fontdict={'family': 'Arial', 'fontsize': 16})
ax.grid('major')
ax.spines['right'].set_color((0.8, 0.8, 0.8))
ax.spines['top'].set_color((0.8, 0.8, 0.8))
handles, labels = ax.get_legend_handles_labels()
handles = handles[:-2] + handles[-2:][::-1]
labels = labels[:-2] + labels[-2:][::-1]
ax.legend(handles, labels, loc='upper right')

plt.title('Decision Curve Analysis')
save_path = 'C:/Users/21581/Desktop/ROCPKL/trainDCA.pdf'
plt.savefig(save_path)

plt.show()
